In [52]:
import re
import math
import numpy

In [6]:
NUM_BP_FINESTRA = 1000
NUM_SEQUENZE = 5
NUM_BASI_MER = 12
PERCENTUALE = 0.1

In [7]:
BASE_CODE = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

In [32]:
BASE_CHAR = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}

In [8]:
f = open('prova.fna')
f.readline()
cromosoma = f.read() #cromosoma senza la prima riga
cromosoma = re.sub("\n","",cromosoma)

In [9]:
def dnaAsByteArray(dnaStr):
    seqLength = len(dnaStr)
    ba = bytearray(math.ceil(seqLength/4))
    for seqPos in range(seqLength):
        ba[seqPos//4] |= BASE_CODE[dnaStr[seqPos]] << (2 * (seqPos%4))
    return ba

In [24]:
def seqAsInt(dnaStr):
    # seqLength = len(dnaStr)
    if len(dnaStr) == 0:
        return 0
    dnaInt = BASE_CODE[dnaStr[0]]
    for dnaBaseChar in dnaStr[1:]:
        dnaInt = dnaInt << 2 | BASE_CODE[dnaBaseChar]
    return dnaInt

In [25]:
seqAsInt('TCC')

53

In [23]:
def kmerCounts(k):
    return bytearray(round(2 ** (math.ceil(k/4) - 2)))

In [25]:
kmerCounts(4)

bytearray(b'')

In [27]:
def appendToSeq(seqAsInt, dnaBaseChar, seqSize):
    return seqAsInt << 2 & (2 ** (seqSize * 2) - 1) | BASE_CODE[dnaBaseChar]

In [31]:
appendToSeq(53,'T',12)

215

In [34]:
def seqToChars(seqAsInt, seqSize):
    seqStr = ''
    for seqPos in range(seqSize):
        seqStr = BASE_CHAR[seqAsInt & 3] + seqStr
        seqAsInt = seqAsInt >> 2
    return seqStr

In [35]:
seqToChars(seqAsInt('ACTGTTCACA'), 10)

'ACTGTTCACA'

In [36]:
seqToChars(seqAsInt('ACTGACTTCACA'), 12)

'ACTGACTTCACA'

In [38]:
seqToChars(appendToSeq(seqAsInt('ACTGACTTCACA'),'C', 12), 12)

'CTGACTTCACAC'

In [73]:
def uniqueKmersRepeatedRatio(seqStr, kmerSize):
#    kmerCounts = [0 for x in range(2 ** (kmerSize * 2))]
    kmerCounts = numpy.zeros((2 ** (kmerSize * 2),), dtype=int)
    uniqueKmersFound = 0
    uniqueKmersRepeated = 0
    if (len(seqStr) < kmerSize):
        return kmerCounts
    currKmer = seqAsInt(seqStr[:kmerSize - 1])
    for seqChar in seqStr[kmerSize - 1:]:
        currKmer = appendToSeq(currKmer, seqChar, kmerSize)
        kmerCounts[currKmer] += 1
        if (kmerCounts[currKmer] == 1):
            uniqueKmersFound += 1 
        if (kmerCounts[currKmer] == 2):
            uniqueKmersRepeated += 1
    return uniqueKmersRepeated / uniqueKmersFound

In [55]:
f = open('chr_21.fasta')
app = f.readline()
cromosoma = f.read() #cromosoma senza la prima riga
cromosoma = re.sub("\n","",cromosoma)
cromosoma = cromosoma.upper()

In [56]:
windowIndexes = range(len(cromosoma)//NUM_BP_FINESTRA)

In [89]:
%%time
#rateOfRepeatedKmersByWindow = []
rateOfRepeatedKmersByWindow = numpy.empty(len(windowIndexes))
for windowIndex in windowIndexes: 
#for windowIndex in range(1000): # range reduced for testing
    windowStart = NUM_BP_FINESTRA * windowIndex
    window = cromosoma[windowStart: min(windowStart + NUM_BP_FINESTRA, len(cromosoma))]
    rateOfRepeatedKmersByWindow[windowIndex] = uniqueKmersRepeatedRatio(window, NUM_BASI_MER)

CPU times: user 1min 44s, sys: 2min 12s, total: 3min 57s
Wall time: 3min 57s


In [180]:
windowWitHighRepetition = numpy.where(rateOfRepeatedKmersByWindow >= 0.1, 1, 0)
startRegionIndices = numpy.where(numpy.diff(windowWitHighRepetition) > 0)[0]
endRegionIndices = numpy.where(numpy.diff(windowWitHighRepetition) < 0)[0]
if windowWitHighRepetition[0] == 1:
    startRegionIndices = numpy.insert(startRegionIndices, 0, 0)
if windowWitHighRepetition[-1] == 1:
    endRegionIndices = numpy.append(endRegionIndices, len(windowWitHighRepetition) - 1)
regionSizes = endRegionIndices - startRegionIndices

In [181]:
numpy.transpose(numpy.vstack((startRegionIndices, endRegionIndices, regionSizes)))

array([[    0,     2,     2],
       [   68,    69,     1],
       [   75,    76,     1],
       ...,
       [44798, 44800,     2],
       [45077, 45078,     1],
       [45085, 45089,     4]])